In [1]:
import os
import pandas as pd

In [16]:
def get_fullpath(filename: str):
    return os.sep.join(['C:','Users','suare','data', 'raw', 'quantquote', 'minutes', filename])

file = 'spy_historical.csv'
# filepath = os.sep.join(['C:','Users','suare','data', 'raw', 'quantquote', 'minutes', file])
filepath = get_fullpath(file)

In [17]:
df = pd.read_csv(filepath)
df.drop(columns=['splits', 'earnings', 'dividends'], errors='ignore', inplace=True)
df.head()

,date,time,open,high,low,close,volume
0,19980102,931,64.9236,64.9236,64.9236,64.9236,97424.60
1,19980102,932,65.0103,65.0103,64.9236,64.9436,31325.70
2,19980102,933,65.0503,65.0503,64.9436,65.0303,32075.20
3,19980102,934,64.9703,64.9703,64.9703,64.9703,1498.84
4,19980102,935,65.0170,65.0303,64.9703,64.9703,17536.40


# 1. Let's split the set in Mahab, Dev and Train sets.

In [5]:
mahab_years = ['2008', '2009']  # add years as str
mahab_subset_df = df[df.date.astype(str).str[:4].isin(mahab_years)]

mahab_months = dict() # one pos per state
mahab_months['state_1'] = ['200810','200811','200812'] # bear trend Q4 2008
mahab_months['state_2'] = ['200904','200905','200906'] # bull trend Q2 2009 
mahab_months['state_3'] = ['200910','200911','200912'] # lateral trend Q4 2009

mahab_state_dfs = dict() # one df per state
for state, months in mahab_months.items():
    mahab_state_dfs[state] = \
        mahab_subset_df[mahab_subset_df.date.astype(str).str[:6].isin(mahab_months[state])]
    mahab_state_dfs[state].to_csv(get_fullpath(f'spy_mahab_{state}.csv'), index=False)

In [23]:
devset_years = ['2010']  # add years as str
devset_subset_df = df[df.date.astype(str).str[:4].isin(devset_years)]
devset_subset_df.to_csv(get_fullpath(f'spy_devset_{"-".join(devset_years)}.csv'), index=False)

In [22]:
from_yr = 2011  # as int
to_yr = 2020    # as int
trainset_years = list(pd.Series(range(from_yr, to_yr + 1, 1)).astype(str))  # add years as str
trainset_subset_df = df[df.date.astype(str).str[:4].isin(trainset_years)]
trainset_subset_df.to_csv(get_fullpath(f'spy_trainset_{from_yr}-{to_yr}.csv'), index=False)

In [20]:
trainset_subset_df.head()

,date,time,open,high,low,close,volume
1245406,20110103,930,105.114,105.138,105.072,105.122,3538420.0
1245407,20110103,931,105.122,105.180,105.097,105.180,1039570.0
1245408,20110103,932,105.180,105.197,105.130,105.147,727576.0
1245409,20110103,933,105.147,105.188,105.130,105.138,554285.0
1245410,20110103,934,105.138,105.155,105.130,105.155,751152.0
